![img](https://i.imgur.com/ecEdqxY.jpg)


Hey there, Eric here. 

I'm pretty new to R & ML in general. I dabbled with openCV in Node.JS but for this I'm winging it. 

If you can dream it you can do it. 

Basically the stategy in this kernel is to use the start and end points from lots of taxi rides and lots of start and end points  [some kind of trip]  from the NYC Department of Transportation. I've made an attempt at a graphic to explain my thought process below (*note: I dropped the time_of_day from this, the DOT data is a floating timestamp).

![](https://i.imgur.com/5zLDiT9.png)

![](https://i.imgur.com/FsKEh1t.png)

As of Sept 14th this is stil a work in progress, if anyone reading this sees any brutal mistakes or if there are simpler ways of doing this please let me know :) 

Any help making document more terse would be appreciated.

**HEADS UP** I haven't actually made the model any better using this datasource. I'm still playing with it. Best score is : 4.05078 (in another kernel, sans seed)

Second Dataset Source: 
https://data.cityofnewyork.us/Transportation/Real-Time-Traffic-Speed-Data/qkm5-nuaq

A bit of research I did: 
https://en.wikipedia.org/wiki/Taximeter https://gis3.dot.ny.gov/html5viewer/?viewer=tdv https://www.autoinsurancecenter.com/traffic-accidents-in-new-york-city.htm https://www.mapsofworld.com/usa/states/new-york/new-york-maps/new-york-lat-long-map.jpg http://www.nyc.gov/html/dot/html/about/mobilityreport.shtml

Website : https://www.casey.works

GitHub : https://www.github.com/EricCasey

Twitch : https://www.twitch.tv/casey_works


**STEP 1** : Load Packages

In [ ]:
print("init")

packs <- c("tidyverse", "lubridate", "nFactors", "stringr", 
           "import", "glmnet", "readr",  "RSNNS", "plyr", 
           "dplyr", "caret", "beepr", "mgcv", "spls", "gbm", 
           "xgboost", "corrplot", "geosphere", "DMwR" ,"MuMIn", 
           "png","grid")

lapply(packs, require, character.only = TRUE)

**STEP 2** : Some general config options

In [ ]:
set.seed(42) # DON'T PANIC
options(na.action = "na.fail")
options(stringsAsFactors = FALSE)

# train_rows_import <- 1000000 
# dot_rows_import   <- 1000
# train_rows_sample <- train_rows_import / 2
# dot_rows_sample   <- dot_rows_import / 1.2

train_rows_import <- 30000
dot_rows_import   <- 1000
train_rows_sample <- train_rows_import / 5
dot_rows_sample   <- dot_rows_import / 5

# train_folds <- 100
train_folds <- 10
train_depth <- 3
train_repeats <- 5
train_tuneLength <- 5
train_trees <- 250

# TODO : "inclusive of tolls"
# TODO : Backwards trip strings?
# TODO : Remove training oints in the water (IN PROGRESS)
# TODO : RUSH HOUR var?
# TODO : HOLIDAYS var?
# TODO : Can I use all points in the line to make routes? mindblown.gif
# TODO : What can I do with those polylines?
# TODO : What about the boroughs?
# TODO : TripStr match rate analysis
# TODO : Add the test data to the training set? make lm, impute fares, train?

**STEP 3** : I think this is filesize

In [ ]:
train_size <- round(file.info("../input/new-york-city-taxi-fare-prediction/train.csv", extra_cols = TRUE)$size / 1000000000, 5)
print(paste("Training Data Size: ", train_size, "gb", sep = ""))
test_size <- round(file.info("../input/new-york-city-taxi-fare-prediction/test.csv", extra_cols = TRUE)$size / 1000000000, 5)
print(paste("Test Data Size: ", test_size, "gb", sep = ""))
dot_size <- round(file.info("../input/dot-file/DOT_Traffic_Speeds_NBE.csv", extra_cols = TRUE)$size / 1000000000, 5)
print(paste("NYC DOT Data Size: ", dot_size, "gb", sep = ""))

**STEP 4** : Import Base Data

In [ ]:
train_dirt <- read.csv("../input/new-york-city-taxi-fare-prediction/train.csv",
                        header = TRUE,
                        colClasses = c(
                            "key"="character",
                            "fare_amount"="numeric",
                            "pickup_datetime"="POSIXct", ## in UTC
                            "dropoff_longitude"="numeric",
                            "pickup_longitude"="numeric",
                            "dropoff_latitude"="numeric",
                            "pickup_latitude"="numeric",
                            "passenger_count"="integer"
                        ), nrows = train_rows_import) #%>% select(-key) # Preparations A through G were complete failures

train_dirt <- dplyr::sample_n(train_dirt, train_rows_sample) # https://www.kaggle.com/docxian/nyc-taxi-get-subsample-of-data
train_raw <- train_dirt

test_dirt <- read.csv("../input/new-york-city-taxi-fare-prediction/test.csv",
                        header = TRUE,
                        colClasses = c(
                            "key"="character",
                            "pickup_datetime"="POSIXct",    ## in UTC
                            "dropoff_longitude"="numeric",
                            "pickup_longitude"="numeric",
                            "dropoff_latitude"="numeric",
                            "pickup_latitude"="numeric",
                            "passenger_count"="integer"
                        )
                    ) #%>% select(-key)

print(names(train_dirt))

**STEP 5** : Define Geographic Scope

In [ ]:
# There's so much data it can't be worth it to keep NAs
dirt_data_train <- na.omit(train_dirt)
dirt_data_test <- na.omit(test_dirt) # * for now

# Remove the lat/lon from the training data that are outside of what's present in the test data
train_dirt <- train_dirt[train_dirt$pickup_latitude < max(test_dirt$pickup_latitude), ]
train_dirt <- train_dirt[train_dirt$pickup_latitude > min(test_dirt$pickup_latitude), ]
train_dirt <- train_dirt[train_dirt$dropoff_latitude < max(test_dirt$dropoff_latitude), ]
train_dirt <- train_dirt[train_dirt$dropoff_latitude > min(test_dirt$dropoff_latitude), ]
train_dirt <- train_dirt[train_dirt$pickup_longitude > min(test_dirt$pickup_longitude), ]
train_dirt <- train_dirt[train_dirt$pickup_longitude < max(test_dirt$pickup_longitude), ]
train_dirt <- train_dirt[train_dirt$dropoff_longitude > min(test_dirt$dropoff_longitude), ]
train_dirt <- train_dirt[train_dirt$dropoff_longitude < max(test_dirt$dropoff_longitude), ]

** STEP 6 ** : Handle Outliers

In [ ]:
# $2.50 starting rate & reasonable limit
dirt_data_train <- dirt_data_train[dirt_data_train$fare_amount >= 2.50, ]  # Minimum $2.50 fare
dirt_data_train <- dirt_data_train[dirt_data_train$fare_amount <= 300, ]  # Cap at $300 fare

# dirt_data_train <- dirt_data_train[dirt_data_train$fare_amount >= 0, ]  # Minimum $0 fare

# https://www.youtube.com/watch?v=R-yQw2UD50U
dirt_data_train <- dirt_data_train[dirt_data_train$passenger_count <= 10, ]
dirt_data_test <- dirt_data_test[dirt_data_test$passenger_count <= 10, ]

# More than one passenger?
dirt_data_train <- dirt_data_train[dirt_data_train$passenger_count >= 1, ] 
dirt_data_test <- dirt_data_test[dirt_data_test$passenger_count >= 1, ]
# dirt_data_train <- dirt_data_train[dirt_data_train$passenger_count >= 0, ] 
# dirt_data_test <- dirt_data_test[dirt_data_test$passenger_count >= 0, ]

train_dirt <- na.omit(train_dirt)

# I used this combo to play with the variables and determine the realistic boundaries above
par(mfrow=c(3,2))
plot(density(train_dirt$fare_amount), main = "fare_amount density") # plots the results
plot(density(train_dirt$passenger_count), main = "passenger_count density")

plot(density(train_dirt$pickup_latitude), main = "pickup latitude density")
plot(density(train_dirt$pickup_longitude), main = "pickup longitude density")

plot(density(train_dirt$dropoff_latitude), main = "dropoff latitude density")
plot(density(train_dirt$dropoff_longitude), main = "dropoff longitude density")

In [ ]:
dirt_data_train <- train_dirt
dirt_data_test <- test_dirt

**STEP 7** : Latitudal & longitudal differences

In [ ]:
# BASE DATA: Var 1 : Latitudal Differences 
latdiff_1 <- abs(dirt_data_train$pickup_latitude) - abs(dirt_data_train$dropoff_latitude)
dirt_data_train$latdiff_1 <- latdiff_1
latdiff_1 <- abs(dirt_data_test$pickup_latitude) - abs(dirt_data_test$dropoff_latitude)
dirt_data_test$latdiff_1 <- latdiff_1
# BASE DATA: Var 2 : Latittudal Differences 
latdiff_2 <- abs(dirt_data_train$pickup_latitude - dirt_data_train$dropoff_latitude)
dirt_data_train$latdiff_2 <- latdiff_2
latdiff_2 <- abs(dirt_data_test$pickup_latitude - dirt_data_test$dropoff_latitude)
dirt_data_test$latdiff_2 <- latdiff_2
# BASE DATA: Var 3 : Longitudal Differences 
longdiff_1 <- abs(dirt_data_train$pickup_longitude) - abs(dirt_data_train$dropoff_longitude)
dirt_data_train$longdiff_1 <- longdiff_1
longdiff_1 <- abs(dirt_data_test$pickup_longitude) - abs(dirt_data_test$dropoff_longitude)
dirt_data_test$longdiff_1 <- longdiff_1
# BASE DATA: Var 3 : Longitudal Differences 
longdiff_2 <- abs(dirt_data_train$pickup_longitude - dirt_data_train$dropoff_longitude)
dirt_data_train$longdiff_2 <- longdiff_2
longdiff_2 <- abs(dirt_data_test$pickup_longitude - dirt_data_test$dropoff_longitude)
dirt_data_test$longdiff_2 <- longdiff_2

print(names(dirt_data_train))

**STEP 8** : Distance & Bearing

In [ ]:
dirt_data_train <- dirt_data_train %>% rowwise() %>% 
                    mutate(crowdist = distHaversine(cbind(pickup_longitude, pickup_latitude), 
                                                    cbind(dropoff_longitude, dropoff_latitude)))

dirt_data_test <- dirt_data_test %>% rowwise() %>% 
    mutate(crowdist = distHaversine(cbind(pickup_longitude, pickup_latitude), 
                                    cbind(dropoff_longitude, dropoff_latitude)))

d_lat1 <- dirt_data_train$pickup_latitude
d_lat2 <- dirt_data_train$dropoff_latitude
d_lon1 <- dirt_data_train$pickup_longitude
d_lon2 <- dirt_data_train$dropoff_longitude

t_lat1 <- dirt_data_test$pickup_latitude
t_lat2 <- dirt_data_test$dropoff_latitude
t_lon1 <- dirt_data_test$pickup_longitude
t_lon2 <- dirt_data_test$dropoff_longitude

d_bearing <- atan2(cos(d_lat1)*sin(d_lat2)-sin(d_lat1)*cos(d_lat2)*cos(d_lon2-d_lon1), 
               sin(d_lon2-d_lon1)*cos(d_lat2)) 

t_bearing <- atan2(cos(t_lat1)*sin(t_lat2)-sin(t_lat1)*cos(t_lat2)*cos(t_lon2-t_lon1), 
               sin(t_lon2-t_lon1)*cos(t_lat2)) 

dirt_data_train$bearing <- d_bearing
dirt_data_test$bearing <- t_bearing

print(names(dirt_data_test))

**STEP 9** : TODO : Airports & Landmarks

In [ ]:
# jfk_coords = (40.639722, -73.778889)
# ewr_coords = (40.6925, -74.168611)
# lga_coords = (40.77725, -73.872611)

# middle_coords = (40.7141667,-74.0063889)

**STEP 10** : The Trip Strings

In [ ]:
dirt_data_train <- dirt_data_train %>% 
mutate("tripStr_3" = paste(format(round(pickup_latitude, digits = 3), nsmall = 3), 
                           format(round(pickup_longitude, digits = 3), nsmall = 3),
                           format(round(dropoff_latitude, digits = 3), nsmall = 3),
                           format(round(dropoff_longitude, digits = 3), nsmall = 3),
                           sep = "|"))

dirt_data_test <- dirt_data_test %>% 
mutate("tripStr_3" = paste(format(round(pickup_latitude, digits = 3), nsmall = 3), 
                           format(round(pickup_longitude, digits = 3), nsmall = 3),
                           format(round(dropoff_latitude, digits = 3), nsmall = 3),
                           format(round(dropoff_longitude, digits = 3), nsmall = 3),
                           sep = "|"))     

dirt_data_train <- dirt_data_train %>% 
mutate("tripStr_2" = paste(format(round(pickup_latitude, digits = 2), nsmall = 2), 
                           format(round(pickup_longitude, digits = 2), nsmall = 2),
                           format(round(dropoff_latitude, digits = 2), nsmall = 2),
                           format(round(dropoff_longitude, digits = 2), nsmall = 2),
                           sep = "|"))

dirt_data_test <- dirt_data_test %>% 
mutate("tripStr_2" = paste(format(round(pickup_latitude, digits = 2), nsmall = 2), 
                           format(round(pickup_longitude, digits = 2), nsmall = 2),
                           format(round(dropoff_latitude, digits = 2), nsmall = 2),
                           format(round(dropoff_longitude, digits = 2), nsmall = 2),
                           sep = "|"))  

dirt_data_train <- dirt_data_train %>% 
mutate("tripStr_1" = paste(format(round(pickup_latitude, digits = 1), nsmall = 1), 
                           format(round(pickup_longitude, digits = 1), nsmall = 1),
                           format(round(dropoff_latitude, digits = 1), nsmall = 1),
                           format(round(dropoff_longitude, digits = 1), nsmall = 1),
                           sep = "|"))

dirt_data_test <- dirt_data_test %>% 
mutate("tripStr_1" = paste(format(round(pickup_latitude, digits = 1), nsmall = 1), 
                           format(round(pickup_longitude, digits = 1), nsmall = 1),
                           format(round(dropoff_latitude, digits = 1), nsmall = 1),
                           format(round(dropoff_longitude, digits = 1), nsmall = 1),
                           sep = "|"))  

dirt_data_train$tripStr_3 <- gsub(" ", "", dirt_data_train$tripStr_3, fixed = TRUE)
dirt_data_test$tripStr_3 <- gsub(" ", "", dirt_data_test$tripStr_3, fixed = TRUE)

dirt_data_train$tripStr_2 <- gsub(" ", "", dirt_data_train$tripStr_2, fixed = TRUE)
dirt_data_test$tripStr_2 <- gsub(" ", "", dirt_data_test$tripStr_2, fixed = TRUE)

dirt_data_train$tripStr_1 <- gsub(" ", "", dirt_data_train$tripStr_1, fixed = TRUE)
dirt_data_test$tripStr_1 <- gsub(" ", "", dirt_data_test$tripStr_1, fixed = TRUE)

print("---v--- 1-digit tripStr ---v---")
print(head(dirt_data_train$tripStr_1,1))
print("---v--- 2-digit tripStr ---v---")
print(head(dirt_data_train$tripStr_2,1))
print("---v--- 3-digit tripStr ---v---")
print(head(dirt_data_train$tripStr_3,1))
print(" ")

print(names(dirt_data_train))

**STEP 11** : *Tempus Fugit

In [ ]:
# DD, MM, YYYY 
dirt_data_train$day <- format(as.Date(dirt_data_train$pickup_datetime), "%d")
dirt_data_train$month <- as.factor(format(as.Date(dirt_data_train$pickup_datetime), "%m"))
dirt_data_train$year <- format(as.Date(dirt_data_train$pickup_datetime), "%Y")

# YYYY-MM-DD String
dirt_data_train$date <- paste(format(as.Date(dirt_data_train$pickup_datetime), "%Y"), 
                              format(as.Date(dirt_data_train$pickup_datetime), "%m"),
                              format(as.Date(dirt_data_train$pickup_datetime), "%d"), sep = "-")

# Hour & Time Of Day
dirt_data_train$hour <- (as.numeric(dirt_data_train$pickup_datetime) %% (24*60*60) / 3600) 
dirt_data_train$tod = cut(dirt_data_train$hour, 
                          breaks = c(0, 4, 9, 15, 18, 24),
                          labels = c("h_early", "h_commute_to", "h_bizhrs", "h_commute_from", "h_night"), 
                          include.lowest=TRUE, right=FALSE)

# Day of Week
dirt_data_train$dow <- as.factor(lubridate::wday(as.Date(dirt_data_train$pickup_datetime)))

# V-- same stuff for test data (I probably didn't need to do it this way)
dirt_data_test$day <- format(as.Date(dirt_data_test$pickup_datetime), "%d")
dirt_data_test$month <- as.factor(format(as.Date(dirt_data_test$pickup_datetime), "%m"))
dirt_data_test$year <- format(as.Date(dirt_data_test$pickup_datetime), "%Y")
dirt_data_test$hour <- (as.numeric(dirt_data_test$pickup_datetime) %% (24*60*60) / 3600)

dirt_data_test$tod = cut(dirt_data_test$hour, 
                         breaks = c(0, 4, 9, 15, 18, 24),
                         labels = c("h_early", "h_commute_to", "h_bizhrs", "h_commute_from", "h_night"), 
                         include.lowest=TRUE, right=FALSE)

dirt_data_test$dow <- as.factor(lubridate::wday(as.Date(dirt_data_test$pickup_datetime)))
dirt_data_test$date <- paste(format(as.Date(dirt_data_test$pickup_datetime), "%Y"), 
                              format(as.Date(dirt_data_test$pickup_datetime), "%m"),
                              format(as.Date(dirt_data_test$pickup_datetime), "%d"), sep = "-")                          

test_raw <- cbind(dirt_data_test) # save this for later

**STEP 12** : Load & Extract a sample of the DOT data then make a Trip String from the first and last points in the polyline.

![](https://i.imgur.com/3kBXoUT.png)

In [ ]:
dot_trips <- read.csv("../input/dot-file/DOT_Traffic_Speeds_NBE.csv",
                        header=TRUE,
                        colClasses=c(
                            "ID" = "character",
                            "SPEED" = "numeric", # probably MPH?
                            "TRAVEL_TIME" = "integer", # probably minutes?
                            "STATUS" = "integer", # DEPRICATED
                            "DATA_AS_OF" = "character", # this is not useable, it's a floating timestamp
                            "LINK_POINTS" = "character", #  "40.683644,-73.72667001  40.68314,-73.72692  40.681365,-73.72806  40.678944,-73.72983  40.678236,-73.73047001"
                            "BOROUGH" = "character", # "Queens" "Brooklyn" "Queens" "Queens"
                            "ENCODED_POLY_LINE" = "character",  # "w_iwFtv~`MbBp@bJbFbN`JlC~BhJtJfFrEdHvFtDhCnW~OtAp@nE~CfB`BjBvBlCxGXlALrB}@xXCby@KpDyBva@"
                            "ENCODED_POLY_LINE_LVLS" = "character" # "BBBBBBBBBBBBBBBBBBBBB" "BBBBB" "BBBBBBB" "BBBBBBB"
                            ), nrows = dot_rows_import) #%>% select(-ID)  # 15,651,405 total rows

# TODO : Add nchar of ENCODED_POLY_LINE_LVLS ? easy, worth a try
# TODO : Use the whole link_points string to generate full routes that can be subset

dot_trips <- dplyr::sample_n(dot_trips, dot_rows_sample)

dirt_dot_trips <- as.data.frame(dot_trips)

print(" --- START STRUCTURE --- ")
print(names(dirt_dot_trips))

dot_keeps <- c("SPEED",	"TRAVEL_TIME", "LINK_POINTS", "BOROUGH", "ENCODED_POLY_LINE", "ENCODED_POLY_LINE_LVLS")
dirt_dot_trips <- as.data.frame(dirt_dot_trips[dot_keeps])

dirt_dot_trips$SPEED <- as.numeric(dirt_dot_trips$SPEED)
dirt_dot_trips$TRAVEL_TIME <- as.numeric(dirt_dot_trips$TRAVEL_TIME)

# New York City Center Coords
ny_lat <- 40.785091
ny_lon <- -73.968285

dirt_dot_trips$tripStr_1 <- 0
dirt_dot_trips$tripStr_2 <- 0
dirt_dot_trips$tripStr_3 <- 0

dirt_dot_trips$str_lat <- word(dirt_dot_trips$LINK_POINTS,1)
dirt_dot_trips$str_lon <- word(dirt_dot_trips$LINK_POINTS,1)
dirt_dot_trips$end_lat <- word(dirt_dot_trips$LINK_POINTS,-1)
dirt_dot_trips$end_lon <- word(dirt_dot_trips$LINK_POINTS,-1)

dirt_dot_trips$str_lat <- strsplit(dirt_dot_trips$str_lat, ",")
dirt_dot_trips$str_lon <- strsplit(dirt_dot_trips$str_lon, ",")
dirt_dot_trips$end_lat <- strsplit(dirt_dot_trips$end_lat, ",")
dirt_dot_trips$end_lon <- strsplit(dirt_dot_trips$end_lon, ",")

dirt_dot_trips$str_lat <- word(dirt_dot_trips$str_lat,1)
dirt_dot_trips$str_lon <- word(dirt_dot_trips$str_lon,2)
dirt_dot_trips$end_lat <- word(dirt_dot_trips$end_lat,1)
dirt_dot_trips$end_lon <- word(dirt_dot_trips$end_lon,2)

dirt_dot_trips$str_lat <- as.numeric(str_extract(dirt_dot_trips$str_lat, "\\-*\\d+\\.*\\d*"))
dirt_dot_trips$str_lon <- as.numeric(str_extract(dirt_dot_trips$str_lon, "\\-*\\d+\\.*\\d*"))
dirt_dot_trips$end_lat <- as.numeric(str_extract(dirt_dot_trips$end_lat, "\\-*\\d+\\.*\\d*"))
dirt_dot_trips$end_lon <- as.numeric(str_extract(dirt_dot_trips$end_lon, "\\-*\\d+\\.*\\d*"))

lat1 <- as.numeric(dirt_dot_trips$str_lat)
lat2 <- as.numeric(dirt_dot_trips$end_lat)
lon1 <- as.numeric(dirt_dot_trips$str_lon)
lon2 <- as.numeric(dirt_dot_trips$end_lon)

dirt_dot_trips$lat1 <- lat1
dirt_dot_trips$lon1 <- lon1
dirt_dot_trips$lat2 <- lat2
dirt_dot_trips$lon2 <- lon2
                             
# Distance & Bearing
dirt_dot_trips <- dirt_dot_trips %>% rowwise() %>% 
                    mutate(crowdist = distHaversine(cbind(lon1, lat1), 
                                                    cbind(lon2, lat2)),
                          bearing = atan2(cos(lat1)*sin(lat2)-sin(lat1)*cos(lat2)*cos(lon2-lon1), 
                               sin(lon2-lon1)*cos(lat2)))

# Trip Strings
dirt_dot_trips$str_lat <- format(round(dirt_dot_trips$str_lat, digits = 3), nsmall = 3)
dirt_dot_trips$str_lon <- format(round(dirt_dot_trips$str_lon, digits = 3), nsmall = 3)
dirt_dot_trips$end_lat <- format(round(dirt_dot_trips$end_lat, digits = 3), nsmall = 3)
dirt_dot_trips$end_lon <- format(round(dirt_dot_trips$end_lon, digits = 3), nsmall = 3)

dirt_dot_trips$tripStr_3 <- paste(dirt_dot_trips$str_lat, 
                                  dirt_dot_trips$str_lon,
                                  dirt_dot_trips$end_lat,
                                  dirt_dot_trips$end_lon,
                                  sep = "|")

dirt_dot_trips$str_lat <- format(round(as.numeric(dirt_dot_trips$str_lat), digits = 2), nsmall = 2)
dirt_dot_trips$str_lon <- format(round(as.numeric(dirt_dot_trips$str_lon), digits = 2), nsmall = 2)
dirt_dot_trips$end_lat <- format(round(as.numeric(dirt_dot_trips$end_lat), digits = 2), nsmall = 2)
dirt_dot_trips$end_lon <- format(round(as.numeric(dirt_dot_trips$end_lon), digits = 2), nsmall = 2)

dirt_dot_trips$tripStr_2 <- paste(dirt_dot_trips$str_lat, 
                                  dirt_dot_trips$str_lon,
                                  dirt_dot_trips$end_lat,
                                  dirt_dot_trips$end_lon,
                                  sep = "|")

dirt_dot_trips$str_lat <- format(round(as.numeric(dirt_dot_trips$str_lat), digits = 1), nsmall = 1)
dirt_dot_trips$str_lon <- format(round(as.numeric(dirt_dot_trips$str_lon), digits = 1), nsmall = 1)
dirt_dot_trips$end_lat <- format(round(as.numeric(dirt_dot_trips$end_lat), digits = 1), nsmall = 1)
dirt_dot_trips$end_lon <- format(round(as.numeric(dirt_dot_trips$end_lon), digits = 1), nsmall = 1)

dirt_dot_trips$tripStr_1 <- paste(dirt_dot_trips$str_lat, 
                                  dirt_dot_trips$str_lon,
                                  dirt_dot_trips$end_lat,
                                  dirt_dot_trips$end_lon,
                                  sep = "|")

dirt_dot_trips <- na.omit(dirt_dot_trips)  # when should I do this?
#print(nrow(dirt_dot_trips))

## Now break the data into three 
dirt_dot_trips_3 <- dirt_dot_trips
dirt_dot_trips_2 <- dirt_dot_trips
dirt_dot_trips_1 <- dirt_dot_trips

## Remove all whitespace
dirt_dot_trips_3$tripStr_3 <- gsub(" ", "", dirt_dot_trips$tripStr_3, fixed = TRUE)
dirt_dot_trips_2$tripStr_2 <- gsub(" ", "", dirt_dot_trips$tripStr_2, fixed = TRUE) 
dirt_dot_trips_1$tripStr_1 <- gsub(" ", "", dirt_dot_trips$tripStr_1, fixed = TRUE)

## THIS MAKES SURE THE TRIP-STRINGS THE RIGHT LENGTH
dirt_dot_trips_1 <- dirt_dot_trips_1[nchar(as.character(dirt_dot_trips_1$tripStr_1)) == 21, ]
dirt_dot_trips_2 <- dirt_dot_trips_2[nchar(as.character(dirt_dot_trips_2$tripStr_2)) == 25, ] 
dirt_dot_trips_3 <- dirt_dot_trips_3[nchar(as.character(dirt_dot_trips_3$tripStr_3)) == 29, ]

dirt_dot_trips_1 <- dirt_dot_trips_1[, c("tripStr_1", "SPEED", "TRAVEL_TIME", "bearing", "crowdist") ]
dirt_dot_trips_2 <- dirt_dot_trips_2[, c("tripStr_2", "SPEED", "TRAVEL_TIME", "bearing", "crowdist") ] 
dirt_dot_trips_3 <- dirt_dot_trips_3[, c("tripStr_3", "SPEED", "TRAVEL_TIME", "bearing", "crowdist") ]

dirt_dot_trips <- na.omit(dirt_dot_trips) 

print(" --- END STRUCTURE --- ")
print(names(dirt_dot_trips))


**STEP 13** : Scatter Maps (WORKING ON REMOVING WATER POINTS)

In [ ]:
# https://www.kaggle.com/nulldata/nyc-map-just-with-taxi-pickup-and-drop-locations
# options(repr.plot.width = 12, repr.plot.height = 10)
library(grid)
#reference: https://gist.github.com/jslefche/eff85ef06b4705e6efbc
library(gridExtra)

# print(paste("Waterworld Rows : ", nrow(dirt_data_train)))

min_lat <- min(test_dirt$pickup_latitude, test_dirt$dropoff_latitude)
max_lat <- max(test_dirt$pickup_latitude, test_dirt$dropoff_latitude)
min_lon <- min(test_dirt$pickup_longitude, test_dirt$dropoff_longitude)
max_lon <- max(test_dirt$pickup_longitude, test_dirt$pickup_longitude)

# (-74.5, -72.8, 40.5, 41.8)
box_0 <- c(min_lon, max_lon, min_lat, max_lat) # TEST SCOPE
box_1 <- c(-74.5, -72.8, 40.5, 41.8)           # 1242x1262
box_2 <- c(-74.3, -73.7, 40.5, 40.9) # 1752 wide x 1538 high - CLOSE UP
box_3 <- c(-74, -73.95, 40.7, 40.8)  # EXTREME CLOSE UP!

boundary <- box_1

map_1 <- na.omit(dirt_data_train)
map_2 <- na.omit(dirt_data_test)
map_3 <- dirt_dot_trips

# print(str(map_1))
# print(head(map_3$lon1))

img_1 <- png::readPNG("../input/carta10/carta10.png") 
img_1_inv <- png::readPNG("../input/carta10inv/carta10inv.png") 
img_1_blk <- png::readPNG("../input/carta12blk/carta10blk.png") #[:,:,0] > 0.9

img_2 <- png::readPNG("../input/carta11/carta11.png") 
img_2_inv <- png::readPNG("../input/carta11inv/carta11inv.png") 
img_2_blk <- png::readPNG("../input/carta11blk/carta11blk.png") 

library(biOps)
img_2_jpg <- biOps::readJpeg("../input/carta11blkjpg/carta11blkjpeg.jpeg")
img_2_mat <- biOps::imagedata(img_2_jpg, type = "grey")

black_map <- as.matrix(img_2_mat)

# print(dim(black_map))

plot(black_map, main = "BLACK LAND - WHITE WATER MAP")

dx <- dim(black_map)[1]
dy <- dim(black_map)[2]

map_1 <- map_1 %>% rowwise() %>% 
                    mutate(xpx = as.integer(dx * (map_1$pickup_longitude - boundary[1]) / (boundary[2]-boundary[1])), 
                           ypx = as.integer(dy - dy * (map_1$pickup_latitude - boundary[3]) / (boundary[4]-boundary[3])))

# map_1$in_water <- black_map[map_1$xpx, map_1$ypx]
# idx <- black_map[head(map_1$xpx, 20), head(map_1$ypx,20)]
# map_1_a <- head(map_1, 200)
# black_map[map_1$xpx, map_1$ypx]
# pickup_xpx <- as.vector(map_1$xpx)
# pickup_ypx <- as.vector(map_1$ypx)

# library(bigmemory)

# black_map <- big.matrix(as.big.matrix(black_map), type = "integer", shared = TRUE, ncol = dx, nrow = dy)

# in_water <- function(x,y) {
#     return(black_map[x,y] > 200)
# }

# map_1$in_water <- in_water(pickup_xpx, pickup_ypx)


# print(str(map_1_a))
# 255 is white
#  lon lon wid . -- lat lat hei
# (-74.5, -72.8,  -- 40.5, 41.8) 
# box_1  = 1752 wide x 1538 height
# point =  40.75719, -73.99724
# .      as.integer(dy - dy * (lat - bound[2]) / (bound[3]-bound[2])) 
# lon -> 1752 * ((-73.99724)-(-74.5)) / ((-72.8)-(-74.5))
#       as.integer(dx * (lon - bound[0]) / (bound[1]-bound[0]))
# lat -> 1538 - (1538 * (40.75719 - 40.5) / (41.8 - 40.5))
# lat_y <- function(lat, dy, bound) {
#     res <- as.integer(dy - dy * (lat - bound[2]) / (bound[3]-bound[2]))   
# }
# lon_x <- function(lon, dx, bound) {
#     res <- as.integer(dx*(lon - bound[0])/(bound[1]-bound[0]))
# }
# result <- apply(head(map_1, 20), 1, isWater(map_1$pickup_longitude, map_1$pickup_latitude, 
#                         dim(black_map)[1], dim(black_map)[2], 
#                         boundary))
# isWater <- function(lon, lat, xpx, ypx, boundary) {
#  #   c(-74.5, -72.8, 40.5, 41.8)
# #     print(paste(lon, lat))
# #     print(paste(xpx, ypx))
# #     print(boundary)
# #     print(img_2_mat[100,100])
#     return(1234)
# }
# print(head(result, 20))
     
# lat_y <- function(lat, dy, bound) {
#     res <- as.integer(dy - dy * (lat - bound[2]) / (bound[3]-bound[2]))
#     return(res)
# }
# lon_x <- function(lon, dx, bound) {
#     res <- as.integer(dx*(lon - bound[0])/(bound[1]-bound[0]))
#     return(res)
# }

g_1 <- rasterGrob(img_1, width=unit(1,"npc"), height=unit(1,"npc"), interpolate = FALSE)
g_1_inv <- rasterGrob(img_1_inv, width=unit(1,"npc"), height=unit(1,"npc"), interpolate = FALSE) 
g_1_blk <- rasterGrob(img_1_blk, width=unit(1,"npc"), height=unit(1,"npc"), interpolate = FALSE) 

g_2 <- rasterGrob(img_2, width=unit(1,"npc"), height=unit(1,"npc"), interpolate = FALSE) 
g_2_inv <- rasterGrob(img_2_inv, width=unit(1,"npc"), height=unit(1,"npc"), interpolate = FALSE) 
g_2_blk <- rasterGrob(img_2_blk, width=unit(1,"npc"), height=unit(1,"npc"), interpolate = FALSE) 

# TRAIN Inverted & ZOOMED
ggplot(map_1, aes(x = pickup_longitude, y = pickup_latitude), margin = FALSE ) +
               annotation_custom(g_2_inv, xmin=box_2[1], xmax=box_2[2], ymin=box_2[3], ymax=box_2[4]) +
               geom_point(color = "red", size = 0.01, alpha = 1/2) +
               geom_point(color = "blue", data = map_1, aes(x = dropoff_longitude, y = dropoff_latitude), size = 0.01, alpha = 1/10) +
               geom_point(color = "orange", data = map_3, aes(x = lon1, y = lat1), size = 0.5, alpha = 1/2) +
               scale_x_continuous(limits = c(box_2[1], box_2[2])) +
               scale_y_continuous(limits = c(box_2[3], box_2[4])) +
               ggtitle("Red Pickups, Blue Dropoffs, Orange Speed Measurement Points")

# grid.arrange(cart_1, cart_2, cart_3, cart_4, cart_5, cart_6, ncol = 2)

# print(paste("Landworld Rows : ", nrow(dirt_data_train)))

https://www.kaggle.com/breemen/nyc-taxi-fare-data-exploration?scriptVersionId=5341553

^ I have some significant progress in this above but I'm hitting memory limits.

**STEP 14** : Compare Trip String columns in the two datasets

In [ ]:
dot_count_1 <- dirt_dot_trips_1 %>% group_by(tripStr_1) %>% dplyr::summarize(count=n())
dot_count_2 <- dirt_dot_trips_2 %>% group_by(tripStr_2) %>% dplyr::summarize(count=n())
dot_count_3 <- dirt_dot_trips_3 %>% group_by(tripStr_3) %>% dplyr::summarize(count=n())

train_count_1 <- dirt_data_train %>% group_by(tripStr_1) %>% dplyr::summarize(count=n())
train_count_2 <- dirt_data_train %>% group_by(tripStr_2) %>% dplyr::summarize(count=n())
train_count_3 <- dirt_data_train %>% group_by(tripStr_3) %>% dplyr::summarize(count=n())

dot_results <- data.frame("tripStr" = c(1:3), 
                          "avg_sample_size" = c(mean(dot_count_1$count), 
                                                mean(dot_count_2$count),
                                                mean(dot_count_3$count)),
                          "unique count" = c(nrow(dot_count_1), 
                                             nrow(dot_count_2),
                                             nrow(dot_count_3))
                      )

train_results <- data.frame("tripStr" = c(1:3), 
                          "avg_sample_size" = c(mean(train_count_1$count), 
                                                mean(train_count_2$count),
                                                mean(train_count_3$count)),
                          "unique count" = c(nrow(train_count_1), 
                                             nrow(train_count_2),
                                             nrow(train_count_3))
                      )

print("------- DEPARTMENT OF TRANSPORTATION ------")
print(dot_results)
print("------------- TAXI DATA -------------------")
print(train_results)
# print("------------- DATE RANGE ------------------")
# print(paste("TEST date range : ", min(dirt_data_test$year) , "-", max(dirt_data_test$year)))
# print(paste("TRAIN date range : ", min(dirt_data_train$year) , "-", max(dirt_data_train$year)))
    
dirt_dot_trips <- na.omit(as.data.frame(dirt_dot_trips))

**STEP 15** : Now aggregeate the DOT dataset so we have average speed and travel_time for each Trip String at each level of resolution.

In [ ]:
dot_keeps <- c("tripStr_1", "tripStr_2", "tripStr_3", "SPEED", "TRAVEL_TIME", "bearing", "crowdist") 

tS_1_keeps <- c("tripStr_1", "SPEED", "TRAVEL_TIME")
tS_2_keeps <- c("tripStr_2", "SPEED", "TRAVEL_TIME")
tS_3_keeps <- c("tripStr_3", "SPEED", "TRAVEL_TIME")

dirt_dot_trips <- dirt_dot_trips[dot_keeps]

dirt_dot_trips_3 <- dirt_dot_trips_3[!grepl("NA", dirt_dot_trips_3$tripStr_3),]
dirt_dot_trips_2 <- dirt_dot_trips_2[!grepl("NA", dirt_dot_trips_2$tripStr_2),]
dirt_dot_trips_1 <- dirt_dot_trips_1[!grepl("NA", dirt_dot_trips_1$tripStr_1),]

dirt_dot_trips_3 <- dirt_dot_trips_3[tS_3_keeps]
dirt_dot_trips_2 <- dirt_dot_trips_2[tS_2_keeps]
dirt_dot_trips_1 <- dirt_dot_trips_1[tS_1_keeps]

agg_dot_clean_3 <- dirt_dot_trips_3 %>%
  group_by(tripStr_3) %>%
  summarise_all(funs(mean(., na.rm = TRUE)))

agg_dot_clean_2 <- dirt_dot_trips_2 %>%
  group_by(tripStr_2) %>%
  summarise_all(funs(mean(., na.rm = TRUE)))

agg_dot_clean_1 <- dirt_dot_trips_1 %>%
  group_by(tripStr_1) %>%
  summarise_all(funs(mean(., na.rm = TRUE)))

agg_dot_clean_1 <- agg_dot_clean_1[tS_1_keeps]
agg_dot_clean_2 <- agg_dot_clean_2[tS_2_keeps]
agg_dot_clean_3 <- agg_dot_clean_3[tS_3_keeps]

# print(colSums(is.na(agg_dot_clean_1)))
# print(head(agg_dot_clean_1))
print(str(agg_dot_clean_1))

psych::describe(agg_dot_clean_1)
psych::describe(agg_dot_clean_2)
psych::describe(agg_dot_clean_3)

**STEP 16** : Now fill new columns in the test & train dataset with their associated average speed and travel_time. (starting with .000, .00, .0)

In [ ]:
## Add Columns to Training Data : tripStr_3
dirt_data_train <- cbind(agg_dot_clean_3[match(dirt_data_train$tripStr_3, agg_dot_clean_3$tripStr_3),2], dirt_data_train)
dirt_data_train <- cbind(agg_dot_clean_3[match(dirt_data_train$tripStr_3, agg_dot_clean_3$tripStr_3),3], dirt_data_train)
##                                tripStr_2
dirt_data_train <- cbind(agg_dot_clean_2[match(dirt_data_train$tripStr_2, agg_dot_clean_2$tripStr_2),2], dirt_data_train)
dirt_data_train <- cbind(agg_dot_clean_2[match(dirt_data_train$tripStr_2, agg_dot_clean_2$tripStr_2),3], dirt_data_train)
##                                tripStr_1
dirt_data_train <- cbind(agg_dot_clean_1[match(dirt_data_train$tripStr_1, agg_dot_clean_1$tripStr_1),2], dirt_data_train)
dirt_data_train <- cbind(agg_dot_clean_1[match(dirt_data_train$tripStr_1, agg_dot_clean_1$tripStr_1),3], dirt_data_train)

## Add Columns to Test Data : tripStr_3
dirt_data_test <- cbind(agg_dot_clean_3[match(dirt_data_test$tripStr_3, agg_dot_clean_3$tripStr_3),2], dirt_data_test)
dirt_data_test <- cbind(agg_dot_clean_3[match(dirt_data_test$tripStr_3, agg_dot_clean_3$tripStr_3),3], dirt_data_test)
##                            tripStr_2
dirt_data_test <- cbind(agg_dot_clean_2[match(dirt_data_test$tripStr_2, agg_dot_clean_2$tripStr_2),2], dirt_data_test)
dirt_data_test <- cbind(agg_dot_clean_2[match(dirt_data_test$tripStr_2, agg_dot_clean_2$tripStr_2),3], dirt_data_test)
##                            tripStr_1
dirt_data_test <- cbind(agg_dot_clean_1[match(dirt_data_test$tripStr_1, agg_dot_clean_1$tripStr_1),2], dirt_data_test)
dirt_data_test <- cbind(agg_dot_clean_1[match(dirt_data_test$tripStr_1, agg_dot_clean_1$tripStr_1),3], dirt_data_test)

# print(names(dirt_data_train))
# print(head(dirt_data_train, 10))
# print(str(dirt_data_train))
# print(colSums(is.na(data_test)))

# Create the tripStr Columns and fill train data with tripStr_3
dirt_data_train$tripStr <- dirt_data_train$tripStr_3
my.na <- is.na(dirt_data_train$tripStr)
dirt_data_train$tripStr[my.na] <- dirt_data_train$tripStr_3[my.na]
#                                                    tripStr_2
dirt_data_train$tripStr <- dirt_data_train$tripStr_2
my.na <- is.na(dirt_data_train$tripStr)
dirt_data_train$tripStr[my.na] <- dirt_data_train$tripStr_2[my.na]
#                                                    tripStr_1
dirt_data_train$tripStr <- dirt_data_train$tripStr_1
my.na <- is.na(dirt_data_train$tripStr)
dirt_data_train$tripStr[my.na] <- dirt_data_train$tripStr_1[my.na]

# Create the tripStr Columns and fill test data with tripStr_3
dirt_data_test$tripStr <- dirt_data_test$tripStr_3
my.na <- is.na(dirt_data_test$tripStr)
dirt_data_test$tripStr[my.na] <- dirt_data_test$tripStr_3[my.na]
#                                                    tripStr_2
dirt_data_test$tripStr <- dirt_data_test$tripStr_2
my.na <- is.na(dirt_data_test$tripStr)
dirt_data_test$tripStr[my.na] <- dirt_data_test$tripStr_2[my.na]
#                                                    tripStr_1
dirt_data_test$tripStr <- dirt_data_test$tripStr_1
my.na <- is.na(dirt_data_test$tripStr)
dirt_data_test$tripStr[my.na] <- dirt_data_test$tripStr_1[my.na]


remaining_train_na <- sum(is.na(dirt_data_train$tripStr))
remaining_train_na <- sum(is.na(dirt_data_train$SPEED))
remaining_test_na <- sum(is.na(dirt_data_test$tripStr))
remaining_test_na <- sum(is.na(dirt_data_test$SPEED))

# print(remaining_train_na)
# print(remaining_test_na)
# print(head(dirt_data_train))
# print(str(dirt_data_train))
# print(names(dirt_data_train))

print(paste(round((1 - sum(is.na(dirt_data_train$SPEED)) / train_rows_sample)*100, 2), 
            "percent tripStr match rate in training data") )
print(paste(round((1 - sum(is.na(dirt_data_test$SPEED)) / nrow(test_raw))*100, 2), 
            "percent tripStr match rate in test data") )

# print(head(dirt_data_test))
# print(str(dirt_data_test))
print(names(dirt_data_test))
print(colSums(is.na(dirt_data_test))) # note we have some duplicate columns...

**STEP 17** : Setup base data for KNN imputation of DOT info for unmatched trip strings

In [ ]:
d_keeps <- c("fare_amount", "longdiff_1", "latdiff_1", "longdiff_2", "latdiff_2", "crowdist", "TRAVEL_TIME", "SPEED", "bearing")
t_keeps <- c("key", "longdiff_1", "latdiff_1", "longdiff_2", "latdiff_2", "crowdist", "TRAVEL_TIME", "SPEED", "bearing")

data_train <- na.omit(as.data.frame(dirt_data_train[d_keeps]))

#print(names(data_test))
#data_test <- as.data.frame(dirt_data_test)
data_test <- as.data.frame(dirt_data_test[t_keeps])
print(names(data_test))

**STEP 18** : Impute the missing SPEED & TRAVEL_TIME variables into the test data

In [ ]:
data_test_speed <- data_test
data_test_time <- data_test

data_test$SPEED <- NA
data_test$TRAVEL_TIME <- NA

print(">>> PRE IMPUTATION <<<")
print("-- str test speed --")
print(str(data_test_speed))
print(colSums(is.na(data_test_speed)))

data_test_speed <- dplyr::select_if(data_test_speed, is.numeric) 
data_test_time <- dplyr::select_if(data_test_time, is.numeric) 

print("imputing...")
print(names(data_test))

# print("speed right before imputation")
# print(names(data_test_speed))
# print(colSums(is.na(data_test_speed)))
# print(colSums(data_test_speed))

# print(head(data_test_speed))
# print(head(data_test_speed$SPEED, 30))

data_test_speed <- DMwR::knnImputation(data_test_speed[, !names(data_test_speed) %in% "SPEED"], scale = TRUE )

data_test_time <- DMwR::knnImputation(data_test_time[, !names(data_test_time) %in% "TRAVEL_TIME"], scale = TRUE)

data_test <- cbind(data_test_speed, data_test)
data_test <- cbind(data_test_time, data_test)

print(">>> IMPUTATION COMPLETE <<<")
print("-- data-test --")
#print(str(data_test))
print(names(data_test))
#print(colSums(is.na(data_test)))

STEP ? : Make all clean data available

In [ ]:
train_keeps <- c("fare_amount", 
                  "TRAVEL_TIME", 
                  "SPEED", 
                  "longdiff_1",
                 "latdiff_1", 
                 "longdiff_2", "latdiff_2",
                 "crowdist", "bearing",
                 "month", "year", 
                 "dow",
                 "tod",
                 "hour", 
                 "passenger_count"
                )

test_keeps <- c("TRAVEL_TIME",
                "SPEED", 
                "longdiff_1", 
                "latdiff_1", "longdiff_2", "latdiff_2",
                "crowdist", "bearing", 
                "dow",
                "tod",
                "month", "year",
                "hour", 
                "passenger_count"
                )

data_test <- cbind(test_raw, data_test)
data_test <- data_test[ , colSums(is.na(data_test)) == 0] ## this makes sure that there are no NA values in the test data

## Go through each row and determine if a value is zero

print(nrow(dirt_data_train))
dirt_data_train <- dirt_data_train[dirt_data_train$crowdist > 0, ]          
print(max(dirt_data_test$hour))
print(min(dirt_data_test$hour))
                
print(" *** END GOAL DATASETS *** ")
# END GOAL : TRAIN DATA WITH MORE COLUMNS
# *************************
data_train <- na.omit(as.data.frame(dirt_data_train[train_keeps]))
data_stack_train <- na.omit(as.data.frame(dirt_data_train[train_keeps]))
# ************************
# END GOAL : TEST DATA WITH MORE COLUMNS
# ************************
data_test <- as.data.frame(data_test[test_keeps])
# *************************

# CHECK
# print(colSums(is.na(data_test)))
# print(nrow(is.na(data_test)))
print(str(data_test))
# summary(data_test)

print("  ** TEST ** ")
print(names(data_test))
print(" ** TRAIN ** ")
print(names(data_train))

psych::describe(data_test)
psych::describe(data_train)

**STEP X** : Principal Component Analysis

In [ ]:
# par(mfrow=c(3,1)) #2x2

allNumeric <- dplyr::select_if(data_train, is.numeric) 

# print(names(allNumeric))

# print(e1071::skewness(allNumeric$crowdist))
# print(e1071::skewness(allNumeric$TRAVEL_TIME))
# print(e1071::skewness(allNumeric$SPEED))
# print(e1071::skewness(allNumeric$bearing))
# print(e1071::skewness(allNumeric$hour))
# print(e1071::skewness(allNumeric$passenger_count))


fit <- princomp(allNumeric, cor = TRUE) # extracting PCs from the correlation matrix 

# summary(fit) # print variance accounted for 
# loadings(fit) # pc loadings 
# fit$scores # the principal components

# Determine Number of Factors to Extract (looks like 2 or 3)
ev <- eigen(cor(allNumeric)) # get eigenvalues
ap <- parallel(subject = nrow(allNumeric), var = ncol(allNumeric),
  rep = 100, cent = 0.05)
nS <- nScree(x = ev$values, aparallel = ap$eigen$qevpea)

plot(fit, type = "lines", main = "scree plot") # scree plot 
biplot(fit, main = "biplot", cex = 0.6)
plotnScree(nS)

**STEP 19** : Correlograms with relaxing significance levels

In [ ]:
allNumeric <- dplyr::select_if(data_train, is.numeric) 
corr <- cor(allNumeric, method = "spearman")
mat <- allNumeric

# # https://medium.swirrl.com/@northernjamie Thanks Jamie
cor.mtest <- function(mat, conf.level = 0.95){
    mat <- as.matrix(mat)
    n <- ncol(mat)
    p.mat <- lowCI.mat <- uppCI.mat <- matrix(NA, n, n)
    diag(p.mat) <- 0
    diag(lowCI.mat) <- diag(uppCI.mat) <- 1
for(i in 1:(n-1)){
    for(j in (i+1):n){
        tmp <- cor.test(mat[,i], mat[,j], conf.level = conf.level)
        p.mat[i,j] <- p.mat[j,i] <- tmp$p.value
        lowCI.mat[i,j] <- lowCI.mat[j,i] <- tmp$conf.int[1]
        uppCI.mat[i,j] <- uppCI.mat[j,i] <- tmp$conf.int[2]
    }
}
return(list(p.mat, lowCI.mat, uppCI.mat))
}
res1 <- cor.mtest(corr,0.95)

par(mfrow=c(3,3))

sig <- c(0.00100, 0.00230, 0.00529, 0.01217, 0.02798, 0.06436, 0.14804, 0.34048, 0.499999999)

for (i in 1:9){
  corrplot(corr, method = "square", order = "hclust", tl.col = "black", 
                     tl.cex = 0.75, p.mat = res1[[1]], sig.level = sig[i],  
                     insig = "pch", pch.cex = 1, main = paste("sig level ", sig[i]), 
                     bg = "white", addrect = 3)
}


In [ ]:
round(corr,3)
pairs(fare_amount ~ ., data = corr)

In [ ]:
# this is where the data should be checked

**STEP 20** : Caret modeling config

In [ ]:
labelName <- 'fare_amount'
predictors <- names(data_train)[names(data_train) != labelName]

print(predictors)

PP <- c('pca', 'nzv', 'medianImpute', 'center', 'scale')
#PP <- c('medianImpute', 'center', 'scale')

train_folds <- train_folds 
train_depth <- train_depth
train_repeats <- train_repeats

myGrid <- expand.grid(alpha = 0:1, lambda = seq(0.0001, 0.02, length = train_tuneLength))

myControl <- trainControl(method = "cv", # cross validation of models (cv, boot, repeated_cv)
                          number = train_folds,
                          summaryFunction = defaultSummary,
                          #summaryFunction = twoClassSummary,
                          #classProbs = TRUE, # IMPORTANT!
                          savePredictions = 'final',
                          verboseIter = FALSE,
                          allowParallel = TRUE
                          #index = myFolds # For Stacks
                          )

**STEP 21** : Setup a df to use to collect results manually

In [ ]:
model_list <- c("lm", "gbm", "xgBoost", "blackboost")
model_results <- as.data.frame(model_list)
model_results$RMSE <- NA
model_results$r2 <- NA
model_results$MAE <- NA

print(model_results)

**MODEL 1 : STRAIGHT LINEAR MODEL**

In [ ]:

# "TRAVEL_TIME", "longdiff_2", "latdiff_2", "crowdist", "bearing",
# "month", "year", "dow","tod","hour", "passenger_count"
# crowdist + latdiff_2 + year + longdiff_2 + dow + tod
lm_1 <- lm(
    fare_amount ~ .
#     longdiff_2 + crowdist
#          +  log(TRAVEL_TIME)
# #                            + bearing
            
#            + latdiff_2
           
# # #                            + month 
#                             + year 
#                             + dow 
#                             + tod                    
# #         + passenger_count
           , data_train)

m_1 <- caret::train(fare_amount ~ .,
                    data = data_train,
                    method = "lm",
                    preProcess = PP,
                    trControl = myControl
                    )

# results <- c("lm", summary(lm_1)$sigma, summary(lm_1)$r.squared, NA)
# model_results <- rbind(model_results, results)

print(caret::varImp(m_1))
# print(na.omit(model_results))
# #attributes(summary(lm_1))

In [ ]:
print(summary(m_1))

**MODEL 2** : STOCHASTIC GRADIENT BOOSTING

In [ ]:
# m_2 <- caret::train(fare_amount ~ crowdist + TRAVEL_TIME + latdiff_2 + year + longdiff_2 + dow + tod,
m_2 <- caret::train(fare_amount ~ .,
    data = data_train,
    method = "gbm",
    metric = "RMSE",
    preProcess = PP,
    trControl = myControl,
    tuneGrid = expand.grid(interaction.depth = train_depth,
                            n.trees = train_trees,  
                            shrinkage = 0.1,
                            n.minobsinnode = 10)
)

In [ ]:
print(caret::varImp(m_2))
print(m_2)
# results <- c("gbm", m_2$results$RMSE, m_2$results$Rsquared, m_2$results$MAE)
# model_results <- rbind(model_results, results)
# print(na.omit(model_results))
# attributes(m_2)
# print(m_2$results$RMSE)

* **MODEL 3** :  XGboost

In [ ]:
# m_3 <- caret::train(fare_amount ~ .,
#     data = data_train,
#     method = "xgbTree",
#     metric = "RMSE",
#     tuneLength = train_tuneLength,
#     tuneGrid = expand.grid(nrounds = 20,
#                             max_depth = train_depth,
#                             eta = 0.05,
#                             gamma = 0.005,
#                             colsample_bytree = 0.75,
#                             min_child_weight = 1,
#                             subsample = 0.5),
#     preProcess = PP,
#     trControl = myControl
# )

In [ ]:
# print(caret::varImp(m_3))
# print(m_3)

# results <- c("xgbTree", m_3$results$RMSE, m_3$results$Rsquared, m_3$results$MAE)
# model_results <- rbind(model_results, results)
# print(na.omit(model_results))

**STEP 4 **: Blackboost Model

In [ ]:
m_4 <- caret::train(fare_amount ~ .,
    data = data_train,
    method = "blackboost",
    metric = "RMSE",
    tuneLength = train_tuneLength,
    tuneGrid = expand.grid(mstop = c(60, 70, 80), 
                           maxdepth = train_depth),
    preProcess = PP,
    trControl = myControl
)

In [ ]:
print(caret::varImp(m_4))
print(m_4)

# results <- c("blackboost", m_4$results$RMSE, m_4$results$Rsquared, m_4$results$MAE)
# model_results <- rbind(model_results, results)
# print(na.omit(model_results))

**STEP X **: asofijasdoif

In [ ]:
# train_keeps <- c("fare_amount", 
#                   "TRAVEL_TIME", 
#                  "SPEED", 
#                  #"longdiff_1",
#                  "latdiff_1", 
#                  "longdiff_2", "latdiff_2",
#                  "crowdist", 
#                  "bearing",
#                   "month", "year", 
#                   "dow",
#                  #"tod",
#                  "hour", 
#                   "passenger_count"
#                 )

# data_stack_train <- data_stack_train[train_keeps]

# #PP <- c('pca', 'nzv', 'center', 'scale')
# PP <- c('pca', 'nzv', 'medianImpute', 'center', 'scale')

# folds <- 20 # 100 has worked best so far
# repeats <- 5
# myFolds <- createFolds(data_train, k = folds)
# myGrid <- expand.grid(alpha = 0:1, lambda = seq(0.0001, 0.02, length = 10))

# myControl <- trainControl(method = "cv", # cross validation of models (cv, boot, repeated_cv)
#                           number = folds,
#                           summaryFunction = defaultSummary,
#                           #classProbs = TRUE, # IMPORTANT!
#                           savePredictions = 'final',
#                           verboseIter = FALSE,
#                           allowParallel = TRUE,
#                           index = createMultiFolds(data_train, k=folds, times=repeats) # For Stacks
#                           )

# print("caretList - model list")
# model_list_1 <- caretEnsemble::caretList(fare_amount ~ .,
#                                        data = data_stack_train,
#                                        trControl = myControl,
#                                        preProcess = PP,
#                                        continue_on_fail = FALSE,
#                                        trace = FALSE,
#                                        methodList = c(
#                                            "xgbTree",
#                                            "blackboost"
# #                                            "penalized",
# #                                             "rlm",
# #                                            "xgbLinear",
# #                                            "ppr"
# # DOESNT WORK Well at this point --> "svmRadial"
# # DOESNT WORK Well at this point --> "glmnet"
# #  DOESNT WORK Well at this point --> "knn"
# #  DOESNT WORK Well at this point --> "gbm"
#                                                   #"earth",
#                                                   #"bagEarth",
#                                                   #"gcvEarth"
#                                                   #"parRF",
#                                                   #"ranger",
#                                                   #"spikeslab",
#                                                   #
#                                         )
#                                        )

# resamples <- resamples(model_list_1)
# stack <- caretEnsemble::caretStack(model_list_1, method = "glmnet")

# print(summary(resamples))
# print(stack)

**Here's where the model comparison chart should go**

In [ ]:
# print(str(model_results))
# # hist(model_results$RMSE, model_results$model_list)
# model_results <- round(na.omit(model_results),2)
# # plot everything
# ggplot(model_results, aes(model_list, RMSE)) +   
#   geom_bar(aes(fill = r2), position = "dodge", stat = "identity")

**SUBMISSION**

In [ ]:
print(colSums(is.na(data_test)))

predictions_1 <- predict(m_1, data_test)
predictions_l1 <- predict(lm_1, data_test)
predictions_2 <- predict(m_2, data_test)
# predictions_3 <- predict(m_3, data_test)
predictions_4 <- predict(m_4, data_test)

# predictions_stack <- predict(stack, data_test)

predictions <- (predictions_1 + predictions_l1 + predictions_2 + predictions_4) / 4
# predictions <- predictions_1

sub <- read.csv("../input/new-york-city-taxi-fare-prediction/sample_submission.csv") %>%  
         mutate(fare_amount = predictions)

print(head(sub, 20))

write_csv(sub, "submission_14th_12.csv", append = FALSE)

In [ ]:
print("SUBMITTING")
beep(sound = 8, expr = NULL)
print(":(")

Well... at least I had fun.